In [1]:
import transformers; transformers.__version__

/home/golem/projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.56.2'

In [2]:
from model import LLM, LLMconfig, BlockConfig

block_config = BlockConfig(
    n_embd=1024,
    pos_emb='rope',
    dropout=0.2,
    attn='gqa',
    n_head=16,
    moe=False,
    up_dim=3072,
    non_linearity='swiglu',
    n_kv_heads=8,   
)

config = LLMconfig(
    vocab_size=50304,
    block_size=2048,
    n_embd=1024,
    pos_emb='rope',
    dropout=0.02,
    n_layer=12,
    norm='rms',
    act_recomp=True,
    CUSTOM_LAYERS=True,
    layer_configs=[block_config]*12
)

my_model = LLM(config)
my_model

Using Activation Recomputation


LLM(
  (tkn_emb): Embedding(50304, 1024)
  (transformer): ModuleDict(
    (drop): Dropout(p=0.02, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): Attention(
          (attn): GQA(
            (c_attn): Linear(in_features=1024, out_features=2048, bias=True)
            (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (attn_dropout): Dropout(p=0.2, inplace=False)
            (resid_dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (ln1): LayerNorm(
          (norm): RMSNorm((1024,), eps=None, elementwise_affine=True)
        )
        (ln2): LayerNorm(
          (norm): RMSNorm((1024,), eps=None, elementwise_affine=True)
        )
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=6144, bias=False)
          (c_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm(
      (norm): RMS

In [5]:
import torch
from transformers import AutoModel, AutoModelForCausalLM
from transformers.models.qwen3.modeling_qwen3   import Qwen3Config,  Qwen3ForCausalLM
from transformers.models.gemma3.modeling_gemma3 import Gemma3Config, Gemma3ForCausalLM 
from transformers.models.gpt2.modeling_gpt2     import GPT2Config,   GPT2LMHeadModel
from transformers.models.llama4.modeling_llama4 import Llama4Config

In [4]:
qw_model:Qwen3ForCausalLM = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
qw_model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [5]:
gm_model:Gemma3ForCausalLM = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")
gm_model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps

In [6]:
oa_model:GPT2LMHeadModel = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
oa_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
fb_model = AutoModel.from_pretrained("facebook/MobileLLM-R1-140M")
fb_model

Llama4TextModel(
  (embed_tokens): Embedding(128256, 576)
  (layers): ModuleList(
    (0-14): 15 x Llama4TextDecoderLayer(
      (self_attn): Llama4TextAttention(
        (q_proj): Linear(in_features=576, out_features=576, bias=False)
        (k_proj): Linear(in_features=576, out_features=192, bias=False)
        (v_proj): Linear(in_features=576, out_features=192, bias=False)
        (o_proj): Linear(in_features=576, out_features=576, bias=False)
        (qk_norm): Llama4TextL2Norm(eps=1e-05)
      )
      (feed_forward): Llama4TextMLP(
        (gate_proj): Linear(in_features=576, out_features=2048, bias=False)
        (up_proj): Linear(in_features=576, out_features=2048, bias=False)
        (down_proj): Linear(in_features=2048, out_features=576, bias=False)
        (activation_fn): SiLU()
      )
      (input_layernorm): Llama4TextRMSNorm((576,), eps=1e-05)
      (post_attention_layernorm): Llama4TextRMSNorm((576,), eps=1e-05)
    )
  )
  (norm): Llama4TextRMSNorm((576,), eps=1e-05)
 

In [8]:
fb_model = AutoModelForCausalLM.from_pretrained("facebook/MobileLLM-R1-140M")
fb_model

Llama4ForCausalLM(
  (model): Llama4TextModel(
    (embed_tokens): Embedding(128256, 576)
    (layers): ModuleList(
      (0-14): 15 x Llama4TextDecoderLayer(
        (self_attn): Llama4TextAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (qk_norm): Llama4TextL2Norm(eps=1e-05)
        )
        (feed_forward): Llama4TextMLP(
          (gate_proj): Linear(in_features=576, out_features=2048, bias=False)
          (up_proj): Linear(in_features=576, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=576, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Llama4TextRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): Llama4TextRMSNorm((576,), eps

In [8]:
del my_model, qw_model, gm_model, oa_model, fb_model

In [9]:
qw_model_base:Qwen3ForCausalLM = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base")
qw_model_base

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [10]:
print(f"{sum(p.numel() for p in qw_model.parameters()):,}")
print(f"{sum(p.numel() for p in qw_model_base.parameters()):,}")

NameError: name 'qw_model' is not defined